[Chain How to Chaining](https://python.langchain.com/docs/modules/chains/how_to/call_methods)

## Different Call Methods

> All classes inherited from `Chain` offer a few ways of running chain logic. The most direct one is by using `__call__`:<br>
> 只要集成了`Chain`最直接的调用就是`__call__`



In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

from langchain.chains import LLMChain

In [5]:
model = ChatOpenAI()

In [6]:
prompt = PromptTemplate.from_template(
    "Tell me a {adjective} joke"
)

In [7]:
llm_chain = LLMChain(prompt=prompt, llm=model)

> By default, `__call__` returns both **the input and output key values.**

In [8]:
llm_chain(inputs={'adjective': "corny"})

{'adjective': 'corny',
 'text': "Why don't scientists trust atoms?\n\nBecause they make up everything!"}

In [10]:
llm_chain("corny")

{'adjective': 'corny',
 'text': "Sure, here's a corny joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!"}

> You can configure it to **only return output key values** by setting `return_only_outputs` to True.

In [9]:
llm_chain("corny", return_only_outputs=True)

{'text': "Sure, here's a corny joke for you:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!"}

1. 检查LLMChain返回几个**output_keys**

In [12]:
llm_chain.output_keys

['text']

2. If the Chain only outputs one output key. you can use `run` method

> <font color=blue>如果仅返回一个output key, 你可以使用**run**方法: 注:放回的是字符串而不是字典</font> <br>

In [13]:
llm_chain.run("corny")

"Why don't scientists trust atoms?\n\nBecause they make up everything!"

In [14]:
llm_chain.run({'adjective': 'corny'})

"Sure, here's a corny joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!"